## 垃圾邮件分类步骤
* 1查看data
* 2分词
* 3向量化
* 4朴素贝叶斯算法描述
* 5结果，查看error

In [34]:
#朴素贝叶斯分类器训练函数

def trainNBO(trainMatrix,trainCategory): #trainMatrix是训练文档列表，trainCategory是训练文档标签列表
    #总文件数
    numTrainDocs = len(trainMatrix)
    #总单词数
    numWords = len(trainMatrix[0]) ##？？0表示第一列吗？
    #侮辱性文件的出现概率
    pAbusive = sum(trainCategory)/float(numTrainDocs) #标签为abusive的文档占总文档的比例
    #构造单词出现次数列表
    #p0num 正常的统计
    #P1Num 侮辱的统计
    
    p0Num =np.ones(numWords) #全1的矩阵
    p1Num = np.ones(numWords)
    
    #整个数据集单词出现的总数，根据样本/实际调查结果调整分母的值
    global p0Denom
    global p1Denom
    p0Denom = 2.0  ##？？？
    p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            #累加辱骂词的频次
            p1Num += trainMatrix[i]
            #对每篇文章的辱骂的频次 进行统计汇总
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i] #标签为normal的文档，单词表的单词出现过多少次
            p0Denom += sum(trainMatrix[i]) #标签为normal的文档中出现过单词的总量
    
    #类别1，即辱骂性文档，词汇表的各个单词出现的概率[log(P(F1/C1)),log(P(F2/C1)),log(P(F3/C1)),...，log(P(Fn/C1))]
    p1Vect = np.log(p1Num/p1Denom)
    #类别0，即正常的文档，词汇表的各个单词出现的概率[log(P(F1/C0)),log(P(F2/C0)),log(P(F3/C0)),...，log(P(Fn/C0))]
    p0Vect = np.log(p0Num/p1Denom)
    return p0Vect,p1Vect,pAbusive  
    

In [35]:
#垃圾邮件分类email harm/spam

In [36]:
#切分文本
def textParse(bigString):
    import re
    #使用正则表达式来切分句子，其中分隔符是除单词、数字外的任意字符串
    listOfTokens = re.split(r'\W*',bigString)
    return[tok.lower()for tok in listOfTokens if len(tok)> 2]
    

In [37]:
def createVocabList(dataSet):
    """    
    获取所有单词的集合
    ：param dataSet：数据集
    return:所有单词的集合（即不含重复元素的单词列表）    
    """
    vocabSet = set([])  #创建空集合
    for document in dataSet:
        #操作符|用于求两个集合的并集
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

def setOfWords2Vec(vocabList,inputSet): #对词语进行向量化处理
    """
    遍历查看该单词是否出现，出现该单词则将该单词置1
    :param vocabList：所有单词集合列表
    :param inputSet:输入数据集
    :return：匹配列表[0,1,0,1…],其中1与0表示词汇表中的单词是否出现在输入的数据集中
    """
    #创建一个和词汇等长的向量，并将元素都设置为0
    returnVec = [0] * len(vocabList) #[0,0,……]
    #遍历文档中的所有单词，如果出现了词汇表中的单词，则将输出的文档向量中的对应值设为1
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word:%s is not in my Vocabulary!" % word)
    return returnVec

In [45]:
#朴素贝叶斯分类函数
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1 = sum(vec2Classify*p1Vec)+np.log(pClass1)
    p0 = sum(vec2Classify*p0Vec)+np.log(1.0-pClass1)
    if p1>p0:
        return 1
    else:
        return 0

In [48]:
def spamTest():
    """
    Desc:
    对贝叶斯垃圾邮件分类器进行自动化处理。
    Args:
    none
    Returns:
    对测试集中每封邮件进行分类，若邮件分类错误，则错误数加1，最后返回总的错误百分比。
    """
    docList = []
    classList = []
    fullText = []
    for i in range(1,26):  #取前26个文件
        #切分，解析数据，并归类为1类别
        wordList = textParse(r'../email/spam/%d.txt'%i)
        docList.append(wordList)
        classList.append(1)
        #切分，解析数据，并归类为0类别
        wordList = textParse(r'../email/ham/%d.txt'%i)
        docList.append(wordList)
        classList.append(0)
    #创建词汇表
    vocabList = createVocabList(docList)
    trainingSet = list(range(50))
    testSet = [] #建立一个空的测试集
    #随机取 10个邮件用来测试
    for i in range(10):
        #random.uniform(x,y)随机生成一个范围x-y的实数
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNBO(np.array(trainMat),np.array(trainClasses))
                
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1

    print('the errorCount is:',errorCount)
    print('the testSet length is:',len(testSet))
    print('the error rate is:',float(errorCount)/len(testSet))   
        

In [49]:
import random
import numpy as np
import math
spamTest()

the errorCount is: 6
the testSet length is: 10
the error rate is: 0.6
